# 01 Pre-Processing

## Initialize Environment
First import all the necessary packages here:

In [ ]:
# Import necessary packages
import os
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import seaborn as sns
import scanpy.external as sce
import matplotlib.pyplot as pl
import anndata2ri
import logging

from matplotlib import colors
from datetime import datetime as dt
from scipy.stats import median_abs_deviation

# Scanpy settings
sc.settings.verbosity = 3
sc.logging.print_header()

Identify the starting directory. Get a timestamp for the run. From the timestamp, derive the resulting output h5ad filename.

In [ ]:
# Name Variables and Settings
fn = "01_"                              # Filename Prefix
sf = "_preprocessing"                   # Filename Suffix
experiment = "Runx3-mut"                # Experiment batch
savedata = True                         # Save data at the end

# Set working directory
os.chdir("/home/dalbao/2023-012-Runx3mutD8scRNA/AlbaoRunx3Manuscript/single_cell/01_preprocessing")

# Determine work location
print("The work location for this notebook is: " + os.getcwd() + "\n")

# Get a timestamp for the start of the run
timestamp = dt.now()
print("This notebook was last run on " + timestamp.strftime("%y-%m-%d %H:%M"))

# Determine the filename for the expected output h5ad
fn = fn + timestamp.strftime("%y-%m-%d-%H-%M")
print("The filename for the AnnData output of this notebook will be:")
print(fn + sf + "_{RPE|MRE}.h5ad")
print("which will be saved in the WORKDIR/h5ad/ folder.")

The work directory is structured to contain a folder named "outs" which itself contains output from the Cell Ranger multi pipeline. "outs" contains demultiplexed data.

In [ ]:
sd_pre= "../../source_data/gem"
sd_post = "/outs/per_sample_outs"

# Path to look inside
target_dir = sd_pre + "1" + sd_post

# List **only folders**
sample_names = [
    d for d in os.listdir(target_dir)
    if os.path.isdir(os.path.join(target_dir, d))
]
sample_names.sort()
print(sample_names)

Go through all the samples in sample_names and then load the data into a new object in Python. This differs from the tutorial in that the different samples which were multiplexed will be loaded and initially pre-processed seprately before concatenating them and performing normalization.

Make one list per GEM.

In [ ]:
samples1 = [] # Array to contain the AnnData for each of the individual groups
samples2 = [] # Array to contain the AnnData for each of the individual groups

### MANUAL ANNOTATION
infection = ["Arm", "Naive", "Arm", "Arm", "Arm", "Arm", "Arm", "Arm", "Arm"]
timepoint = ["Day 8", "Naive", "Day 8", "Day 8", "Day 5", "Day 8", "Day 8", "Day 8", "Day 8"]

# Loop through all samples in sample_names for sample1
for index in range(0, len(sample_names)):
    
    # Load samples individually into each array element
    samples1.append(sc.read_10x_mtx(sd_pre +
                                    "1" + 
                                    sd_post +
                                    "/" + 
                                    sample_names[index] + 
                                    "/count/sample_filtered_feature_bc_matrix"))

    # Annotate each sample with the experiment as a group observation
    samples1[index].obs["experiment"] = experiment
    # Annotate each sample with the sample name as a group observation
    samples1[index].obs["group"] = sample_names[index]
    # Annotate each sample with the timepoint as a group observation
    samples1[index].obs["timepoint"] = infection[index]
    # Annotate each sample with the infection as a group observation
    samples1[index].obs["infection"] = timepoint[index]
    # Check output
    samples1[index]
# End loop

del(index) # Cleanup

# Loop through all samples in sample_names for sample1
for index in range(0, len(sample_names)):
    
    # Load samples individually into each array element
    samples2.append(sc.read_10x_mtx(sd_pre +
                                    "2" + 
                                    sd_post + 
                                    "/" + 
                                    sample_names[index] + 
                                    "/count/sample_filtered_feature_bc_matrix"))

    # Annotate each sample with the experiment as a group observation
    samples2[index].obs["experiment"] = experiment
    # Annotate each sample with the sample name as a group observation
    samples2[index].obs["group"] = sample_names[index]
    # Annotate each sample with the timepoint as a group observation
    samples2[index].obs["timepoint"] = infection[index]
    # Annotate each sample with the infection as a group observation
    samples2[index].obs["infection"] = timepoint[index]
    # Check output
    samples2[index]
# End loop

del(index) # Cleanup

Then combine all hashtagged samples into one adata file per GEM:

In [ ]:
# Set the first element in the array as the base
adata1 = samples1[0]

# Concatenate sample on top of the base
# Start from sample 1, since adata already contains sample 0 
for index in range(1, len(sample_names)):
    adata1 = ad.concat([adata1, samples1[index]], join = "outer") # Outer option does a union of all genes
# End loop

# Confirm resulting AnnData file then cleanup
print(adata1)
del(samples1, index)

# Set the first element in the array as the base
adata2 = samples2[0]

# Concatenate sample on top of the base
# Start from sample 1, since adata already contains sample 0 
for index in range(1, len(sample_names)):
    adata2 = ad.concat([adata2, samples2[index]], join = "outer") # Outer option does a union of all genes
# End loop

# Confirm resulting AnnData file then cleanup
print(adata2)
del(samples2, index)

In [ ]:
# Make cell_id column
adata1.obs['cell_id'] = adata1.obs.index.astype(str)
adata2.obs['cell_id'] = adata2.obs.index.astype(str)

## Basic pre-processing

For basic quality control metrics, plot the highest expressed genes per sample:

In [ ]:
# adata1
# Plot highest expressed genes per sample
for group in sample_names:
    
    plot = sc.pl.highest_expr_genes(adata1[adata1.obs.group == group, :], n_top=20, show = False)
    plot.set_title(group + " GEM1 Pre-clean")
# End of loop

del(plot) # Cleanup``

In [ ]:
# adata2
# Plot highest expressed genes per sample
for group in sample_names:
    
    plot = sc.pl.highest_expr_genes(adata2[adata2.obs.group == group, :], n_top=20, show = False)
    plot.set_title(group  + " GEM2 Pre-clean")
# End of loop

del(plot) # Cleanup

## Filtering

Create filters for outliers based on counts and number of genes expressed and filter genes based on number of cells or counts.

But first, check the original metric

In [ ]:
print(adata1)
print(adata2)

Compute quality metrics:

In [ ]:
# adata1
# Mitochondrial genes
adata1.var["mt"] = adata1.var_names.str.startswith("mt-")
# Ribosomal genes
adata1.var["ribo"] = adata1.var_names.str.startswith(("Rps", "Rpl"))

# Calculate percent mitochondrial gene contamination
sc.pp.calculate_qc_metrics(adata1, qc_vars=['mt', 'ribo'], percent_top=None, log1p=True, inplace=True)

# Plot quality metrics:

sc.pl.violin(adata1, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], groupby= "group", jitter=0.4, multi_panel=True)
for group in sample_names:
    
    sc.pl.scatter(adata1[adata1.obs.group == group, :], x='total_counts', y='pct_counts_mt', title = "Pre-filter GEM1 "+group)
    sc.pl.scatter(adata1[adata1.obs.group == group, :], x='total_counts', y='n_genes_by_counts', title = "Pre-filter GEM1 "+group)
# End of loop

del(group) # Cleanup

In [ ]:
# adata2
# Mitochondrial genes
adata2.var["mt"] = adata2.var_names.str.startswith("mt-")
# Ribosomal genes
adata2.var["ribo"] = adata2.var_names.str.startswith(("Rps", "Rpl"))

# Calculate percent mitochondrial gene contamination
sc.pp.calculate_qc_metrics(adata2, qc_vars=['mt', 'ribo'], percent_top=None, log1p=True, inplace=True,)

# Plot quality metrics:

sc.pl.violin(adata2, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], groupby= "group", jitter=0.4, multi_panel=True)
for group in sample_names:
    
    sc.pl.scatter(adata2[adata2.obs.group == group, :], x='total_counts', y='pct_counts_mt', title = "Pre-filter GEM2 "+group)
    sc.pl.scatter(adata2[adata2.obs.group == group, :], x='total_counts', y='n_genes_by_counts', title = "Pre-filter GEM2 "+group)
# End of loop

del(group) # Cleanup

Now for each sample in the array "samples" do the following:

1) Correct for ambient RNA.
2) Do doublet discrimination with scrublet and scDblFinder and mark doublets, but do not filter yet!
3) Create low-level filters.
4) High-level filtering:
 - remove cells with less than 200 genes
 - genes expressed in less than 3 cells
 - genes with less than 5 counts
5) Do low level filtering by removing high MT contamination, doublets and low counts.

### Ambient RNA correction on GEM1

First import packages to run R on the notebook:

In [ ]:
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

SoupX requires basic clustering data and raw data, so do that:

In [ ]:
# Make adata copy then normalize
adata_pp = adata1.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)

# Do dimensionality reduction
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")

# Extract groups
soupx_groups = adata_pp.obs["soupx_groups"]

# Cleanup, remove extra adata
del adata_pp

# Prepare data for SoupX
cells = adata1.obs_names
genes = adata1.var_names
data = adata1.X.T

# Import raw data needed for SoupX
adata_raw = sc.read_10x_h5(filename="../../source_data/gem1/outs/multi/count/raw_feature_bc_matrix.h5")
adata_raw.var_names_make_unique()
data_tod = adata_raw.X.T
del adata_raw

Call the R function:

In [ ]:
%%R -i data -i data_tod -i genes -i cells -i soupx_groups -o out 
library(SoupX)

# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SboupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

Take the outputs from R and reassign to adata:

In [ ]:
adata1.layers["counts"] = adata1.X
adata1.layers["soupX_counts"] = out.T
adata1.X = adata1.layers["soupX_counts"]

# Cleanup
del(out, data, data_tod, genes, cells, soupx_groups)

### Ambient RNA correction on GEM2

First import packages to run R on the notebook:

In [ ]:
import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

SoupX requires basic clustering data and raw data, so do that:

In [ ]:
# Make adata copy then normalize
adata_pp = adata2.copy()
sc.pp.normalize_per_cell(adata_pp)
sc.pp.log1p(adata_pp)

# Do dimensionality reduction
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp)
sc.tl.leiden(adata_pp, key_added="soupx_groups")

# Extract groups
soupx_groups = adata_pp.obs["soupx_groups"]

# Cleanup, remove extra adata
del adata_pp

# Prepare data for SoupX
cells = adata2.obs_names
genes = adata2.var_names
data = adata2.X.T

# Import raw data needed for SoupX
adata_raw = sc.read_10x_h5(filename="../../source_data/gem2/outs/multi/count/raw_feature_bc_matrix.h5")
adata_raw.var_names_make_unique()
data_tod = adata_raw.X.T
del adata_raw

Call the R function:

In [ ]:
%%R -i data -i data_tod -i genes -i cells -i soupx_groups -o out 
library(SoupX)

# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data), est = rowSums(data)/sum(data), counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SboupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

Take the outputs from R and reassign to adata:

In [ ]:
adata2.layers["counts"] = adata2.X
adata2.layers["soupX_counts"] = out.T
adata2.X = adata2.layers["soupX_counts"]

# Cleanup
del(out, data, data_tod, genes, cells, soupx_groups)

### Doublet Discrimination

Analyze using scrublet and scDblFinder:

In [ ]:
# Doublet discrimination with scrublet
sce.pp.scrublet(
    adata1,
    adata_sim = None,
    threshold=0.25,
)
%matplotlib inline
sce.pl.scrublet_score_distribution(adata1)

In [ ]:
# Doublet discrimination with scrublet
sce.pp.scrublet(
    adata2,
    adata_sim = None,
    threshold=0.25,
)
sce.pl.scrublet_score_distribution(adata2)

Do another doublet discrimination technique using scDblDinder:

In [ ]:
# Doublet discrimination by scDblFinder
# First extract matrix:
data_mat = adata1.X.T

Import R functions to score doublets:

In [ ]:
%%R -i data_mat -o doublet_score -o doublet_class

library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)


set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
# Reassign scDblFinder scores into data:
adata1.obs["scDblFinder_score"] = doublet_score
adata1.obs["scDblFinder_class"] = doublet_class
adata1.obs.scDblFinder_class.value_counts()

del(doublet_score, doublet_class, data_mat)

In [ ]:
# Doublet discrimination by scDblFinder
# First extract matrix:
data_mat = adata2.X.T

Import R functions to score doublets:

In [ ]:
%%R -i data_mat -o doublet_score -o doublet_class

library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)


set.seed(123)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
doublet_score = sce$scDblFinder.score
doublet_class = sce$scDblFinder.class

In [ ]:
# Reassign scDblFinder scores into data:
adata2.obs["scDblFinder_score"] = doublet_score
adata2.obs["scDblFinder_class"] = doublet_class
adata2.obs.scDblFinder_class.value_counts()

del(doublet_score, doublet_class, data_mat)

In [ ]:
# Restore Plotting
%matplotlib inline

### Combine adata1 and adata2

In [ ]:
# adata1 and adata2 are from two GEMs, merge them together
adata = sc.AnnData.concatenate(adata1, adata2, join = "outer", batch_key="gem", batch_categories=["gem1", "gem2"], index_unique="-")
print(adata)
del(adata1, adata2)

### Create low-level filters

In [ ]:
# Create Filters
conditions = [
    (adata.obs['predicted_doublet'] == True),
    (adata.obs['scDblFinder_class'] == "doublet"),
    (adata.obs['n_genes_by_counts'] < 1250),
    (adata.obs['total_counts'] < 3000),
    (adata.obs['pct_counts_mt'] > 5),
    (adata.obs['pct_counts_mt'] <= 5) & (adata.obs['n_genes_by_counts'] >= 1250) & (adata.obs['total_counts'] >= 3000) & (adata.obs['scDblFinder_class'] == "singlet") & (adata.obs['predicted_doublet'] == False)]

values = [ 
            'Doublet_Scrublet', 
            'Doublet_scDblFinder', 
            'Low_nFeature', 
            'Low_counts', 
            'High_MT', 
            'Pass']
adata.obs['QC'] = np.select(conditions, values)
adata.obs['QC'] = adata.obs['QC'].astype('category')

# Filters
adata.obs['QC'].value_counts()

### Filter

In [ ]:
# High level filtering
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.filter_genes(adata, min_counts=5)

# Filtering
adata = adata[adata.obs['QC'] == 'Pass', :]

## Post-Filtering

Check sample names:

In [ ]:
# Check if sample labeling carried over
print(adata)
print(adata.obs['group'].value_counts())

Check the resulting quality metrics of the combined samples:

In [ ]:
# Plot combined QC metrics, grouped by sample (singlet filtered)

sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], groupby= "group", jitter=0.4, multi_panel=True)
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color = "group", title = "Post-filter All Samples")
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color = "group", title = "Post-filter All Samples")

In [ ]:
# Plot highest expressed genes per sample
for group in sample_names:
    plot = sc.pl.highest_expr_genes(adata[adata.obs.group == group, :], n_top=20, show = False)
    plot.set_title(group + " clean all samples")
# End of loop

del(plot) # Cleanup

## Cell Cycle Regression

These steps socre cell cycle effects and regress these effects out.

In [ ]:
# Load S genes
s_genes =  [x.strip() for x in open('../../source_data/mouse-s-gene-list.csv')]
print(s_genes)

# Load G2M genes
g2m_genes =  [x.strip() for x in open('../../source_data/mouse-g2m-gene-list.csv')]
print(g2m_genes)

# Score cells for cell cycle
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes) # Score filtered adata

## Normalization

Steps from here and onwards describe normalization steps prior to clustering analyses.

The first step is to normalize cell counts to 20,000 reads per cell. We choose this number based on the Day 5 sample distribution.

In [ ]:
# Normalize read counts
sc.pp.normalize_total(adata, target_sum=2e4) # cell filtered

Then logarithmize the data:

In [ ]:
# Change to logarithmic scale
sc.pp.log1p(adata) # cell filtered

Stratify between MRE (adata) and RPE.

Then identify highly-variable genes and plot:

In [ ]:
# Keep only rescue data
RPE   = adata[ (adata.obs.group != "dAD") & 
               (adata.obs.group != "dID") &
               (adata.obs.group != "dVWRPY")].copy()

# Identify variable genes
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pp.highly_variable_genes(RPE, min_mean=0.0125, max_mean=3, min_disp=0.5)
# Plot variable genes
sc.pl.highly_variable_genes(adata)
sc.pl.highly_variable_genes(RPE)

Set the raw attribute of AnnData as the normalized values, which is done prior to regression of count effects. This allows recovery of raw normalized data prior to correction:

In [ ]:
# Store raw data
adata.raw = adata
RPE.raw = RPE

Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance. In this step, I skipped filtering by highly variable genes that is present in the Scanpy tutorial, as I am interested in all genes.

In [ ]:
# Regress out the effects of cell count and mitochondrial contamination
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt', 'S_score', 'G2M_score'])
sc.pp.regress_out(RPE, ['total_counts', 'pct_counts_mt', 'S_score', 'G2M_score'])
# Lastly, scale the data
sc.pp.scale(adata)
sc.pp.scale(RPE)
# Check the resulting AnnData object
print("MRE:")
print(adata)
print("RPE:")
print(RPE)

## Principal Component Analysis

We can now do the preliminary steps prior to clustering analyses since the data is normalized and regressed. Start by identifying principal components:

In [ ]:
# Compute principal components
sc.tl.pca(adata, svd_solver='arpack', n_comps = 75)
sc.tl.pca(RPE, svd_solver='arpack', n_comps = 75)

In [ ]:
# Plot
sc.pl.pca(adata, color='group', return_fig = True, title = "Filtered MRE PCA")
sc.pl.pca(RPE, color='group', return_fig = True, title = "Filtered RPE PCA")

Identify the contribution of each principal component to variance through a Scree plot:

In [ ]:
# Plot the Scree plot:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 60)
sc.pl.pca_variance_ratio(RPE, log=True, n_pcs = 60)

## Save Data
Procedures for saving data are outlined below:

In [ ]:
# Reminder for the saved data
if savedata:

    print("The AnnData outputs of this notebook will be saved in the h5ad/ folder.")

    # Save data to an AnnData file
    print("MRE:" + fn + sf + "_MRE.h5ad")
    adata.write_h5ad(filename = "../../h5ad/" + fn + sf + "_MRE.h5ad", compression = "gzip", compression_opts = 9)
    print(adata)
    print("RPE:" + fn + sf + "_RPE.h5ad")
    RPE.write_h5ad(filename = "../../h5ad/" + fn + sf + "_RPE.h5ad", compression = "gzip", compression_opts = 9)
    print(RPE)

else:

    print("Not saving the AnnData file!")
    
# End of Notebook
print("\nNotebook Ends")